## EvoPlay序列优化生成

### 导入相关package

In [1]:
from __future__ import print_function
import random
import numpy as np
import pandas as pd
from collections import defaultdict, deque
from sequence_env_m_p import Seq_env, Mutate
from mcts_alphaZero_mutate_expand_m_p_gfp import MCTSMutater
# from p_v_net_torch import PolicyValueNet  # Pytorch
from p_v_net_3 import PolicyValueNet
from env_model import CNN2
import torch
import torch.utils.data as data
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn.functional as F
from typing import List, Union
import sys
import datetime
import os

/home/Htang/miniconda3/envs/lm_gvp/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:


def string_to_one_hot(sequence: str, alphabet: str) -> np.ndarray:

    out = np.zeros((len(sequence), len(alphabet)))
    for i in range(len(sequence)):
        out[i, alphabet.index(sequence[i])] = 1
    return out
class MyDataset(data.Dataset):
    def __init__(self, sequences, labels):
        self.sequences = sequences
        self.labels = labels

    def __getitem__(self, index):
        seq, target = self.sequences[index], self.labels[index]
        return seq, target

    def __len__(self):
        return len(self.sequences)
def one_hot_to_string(
    one_hot: Union[List[List[int]], np.ndarray], alphabet: str
) -> str:
    """
    Return the sequence string representing a one-hot vector according to an alphabet.

    Args:
        one_hot: One-hot of shape `(len(sequence), len(alphabet)` representing
            a sequence.
        alphabet: Alphabet string (assigns each character an index).

    Returns:
        Sequence string representation of `one_hot`.

    """
    residue_idxs = np.argmax(one_hot, axis=1)
    return "".join([alphabet[idx] for idx in residue_idxs])

def raw_to_features(data_dir,part=1):
    fr = open(data_dir, "r")
    ll = fr.readlines()
    seq_list = []
    label_list = []
    print("part:{}".format(part))
    for i in range(1, len(ll)):
        tmp_list = ll[i].strip().split("\t")
        seq_list.append(tmp_list[0])
        label_list.append(float(tmp_list[1]))
        
    tem_part = int((len(seq_list)/10)*part)
    
    seq_list = seq_list[:tem_part]
    label_list =label_list[:tem_part]
    seq_np = np.array(
        [string_to_one_hot(seq, AAS) for seq in seq_list]
    )

    labels = torch.from_numpy(np.array(label_list))
    labels = labels.to(torch.float32)
    one_hots = torch.from_numpy(seq_np)
    one_hots = one_hots.to(torch.float32)
    return one_hots, labels

def train_cnn_predictor(data_dir,part=1):
    
    one_hots, labels = raw_to_features(data_dir,part)
    seq_dataset = MyDataset(one_hots, labels)
    index = [8,8,10,10,10,10,5,5,5,5,5]
    epochs = index[part]
    train_loader = DataLoader(
        seq_dataset, batch_size=128, shuffle=True)
    
    model = CNN2(
        len(pab1_wt_sequence),
        len(AAS),).to('cuda')
    optimizer = optim.Adam(model.parameters())
    
    for epoch in range(epochs):
        for i, batch in enumerate(train_loader):
            inputs = batch[0]
            inputs = inputs.permute(0,2,1).to('cuda')
            labels = batch[1].to('cuda')
            optimizer.zero_grad()
            
            logits = model(inputs)
            logits = logits.squeeze()
            loss = F.mse_loss(logits, labels)
            loss.backward()
            optimizer.step()
    print("train_loss:{}".format(loss))
    return model
class TrainPipeline():
    def __init__(self, start_seq, alphabet, model, trust_radius, init_model=None): #init_model=None
        self.seq_len = len(start_seq)
        self.vocab_size = len(alphabet)
        self.n_in_row = 4
        self.seq_env = Seq_env(
            self.seq_len,
            alphabet,
            model,
            start_seq,
            trust_radius)  #n_in_row=self.n_in_row
        self.mutate = Mutate(self.seq_env)
        # training params
        self.learn_rate = 2e-3
        self.lr_multiplier = 1.0  # adaptively adjust the learning rate based on KL
        self.temp = 1.0  # the temperature param
        self.n_playout = 200  # num of simulations for each move 400   1600
        self.c_puct = 10 #0.5  # 10
        self.buffer_size = 10000
        self.batch_size = 32  # mini-batch size for training  512
        self.data_buffer = deque(maxlen=self.buffer_size)
        self.play_batch_size = 1
        self.epochs = 5  # num of train_steps for each update
        self.kl_targ = 0.02
        self.check_freq = 50
        self.game_batch_num = 1500
        self.best_win_ratio = 0.0
        # num of simulations used for the pure mcts, which is used as
        # the opponent to evaluate the trained policy
        self.pure_mcts_playout_num = 1000
        #self_added
        self.buffer_no_extend = False
        #self_added
        #playout
        self.generated_seqs = []
        self.fit_list = []
        self.p_dict = {}
        self.m_p_dict = {}
        self.retrain_flag = False
        self.part = 2
        #playout
        #
        if init_model:
            # start training from an initial policy-value net
            self.policy_value_net = PolicyValueNet(self.seq_len,
                                                   self.vocab_size,
                                                   model_file=init_model,use_gpu=True)
        else:
            # start training from a new policy-value net
            self.policy_value_net = PolicyValueNet(self.seq_len,
                                                   self.vocab_size,use_gpu=True)
        self.mcts_player = MCTSMutater(self.policy_value_net.policy_value_fn,
                                      c_puct=self.c_puct,
                                      n_playout=self.n_playout,
                                      is_selfplay=1)

    def collect_selfplay_data(self, n_games=1):
        """collect self-play data for training"""
        counts = len(self.generated_seqs)
        self.buffer_no_extend = False
        for i in range(n_games):
            play_data, seq_and_fit, p_dict = self.mutate.start_mutating(self.mcts_player,
                                                          temp=self.temp)    #winner,
            play_data = list(play_data)[:]
            self.episode_len = len(play_data)

            self.p_dict = p_dict
            self.m_p_dict.update(self.p_dict)
            if self.episode_len == 0:
                self.buffer_no_extend = True
            else:
                self.data_buffer.extend(play_data)
                for seq, fit in seq_and_fit:  #alphafold_d
                    if seq not in self.generated_seqs:
                        self.generated_seqs.append(seq)
                        self.fit_list.append(fit)
                        if seq not in self.m_p_dict.keys():
                            self.m_p_dict[seq] = fit
                    
                        if len(self.generated_seqs)%10==0 and len(self.generated_seqs)>counts and self.part<=10:
                            self.retrain_flag=True
                       

    def policy_update(self):
        """update the policy-value net"""
        mini_batch = random.sample(self.data_buffer, self.batch_size)
        state_batch = [data[0] for data in mini_batch]
        mcts_probs_batch = [data[1] for data in mini_batch]
        winner_batch = [data[2] for data in mini_batch]
        old_probs, old_v = self.policy_value_net.policy_value(state_batch)
        for i in range(self.epochs):
            loss, entropy = self.policy_value_net.train_step(
                    state_batch,
                    mcts_probs_batch,
                    winner_batch,
                    self.learn_rate*self.lr_multiplier)
            new_probs, new_v = self.policy_value_net.policy_value(state_batch)
            kl = np.mean(np.sum(old_probs * (
                    np.log(old_probs + 1e-10) - np.log(new_probs + 1e-10)),
                    axis=1)
            )
            if kl > self.kl_targ * 4:  # early stopping if D_KL diverges badly
                break
        # adaptively adjust the learning rate
        if kl > self.kl_targ * 2 and self.lr_multiplier > 0.1:
            self.lr_multiplier /= 1.5
        elif kl < self.kl_targ / 2 and self.lr_multiplier < 10:
            self.lr_multiplier *= 1.5

        explained_var_old = (1 -
                             np.var(np.array(winner_batch) - old_v.flatten()) /
                             np.var(np.array(winner_batch)))
        explained_var_new = (1 -
                             np.var(np.array(winner_batch) - new_v.flatten()) /
                             np.var(np.array(winner_batch)))
        print(("kl:{:.5f},"
               "lr_multiplier:{:.3f},"
               "loss:{},"
               "entropy:{},"
               "explained_var_old:{:.3f},"
               "explained_var_new:{:.3f}"
               ).format(kl,
                        self.lr_multiplier,
                        loss,
                        entropy,
                        explained_var_old,
                        explained_var_new))
        return loss, entropy


    def run(self):
        """run the training pipeline"""
        starttime = datetime.datetime.now() 
        #part =2
        try:
            for i in range(self.game_batch_num):
                self.collect_selfplay_data(self.play_batch_size)
                print("batch i:{}, episode_len:{}".format(
                        i+1, self.episode_len))
                if self.retrain_flag and self.part<=10:
                    print('train predictor again')

                    update_model = train_cnn_predictor(data_dir,self.part)
                    
                    self.seq_env.model = update_model
                    self.seq_env.model.eval()
                    self.part = self.part+1
                    self.retrain_flag = False
                if len(self.m_p_dict.keys()) >= 4000:
                    m_p_fitness = np.array(list(self.m_p_dict.values()))
                    m_p_seqs = np.array(list(self.m_p_dict.keys()))
                    df_m_p = pd.DataFrame(
                        {"sequence": m_p_seqs, "pred_fit": m_p_fitness})
                    df_m_p.to_csv(r"./evozero_pab1_generated_sequence_1.csv",index=False)
                    endtime = datetime.datetime.now() 
                    print('time cost：',(endtime-starttime).seconds)
                    sys.exit(0)
                if len(self.data_buffer) > self.batch_size and self.buffer_no_extend == False:
                    loss, entropy = self.policy_update()
        except KeyboardInterrupt:
            print('\n\rquit')

In [3]:

cmd = "nvidia-smi -q -d Memory |grep -A4 GPU|grep Free"
print('cmd: ', cmd)
process = os.popen(cmd) # return file
output = process.read()
print(output)
process.close()
memory_gpu=[int(x.split()[2]) for x in output.strip().split('\n')]
print([str(ii)+':'+str(v) for ii, v in enumerate(memory_gpu)])
print('max remain memory GPU:', str(np.argmax(memory_gpu)))
# set gpu env
os.environ['CUDA_VISIBLE_DEVICES']=str(np.argmax(memory_gpu))


data_dir = '/home/Htang/workspace/project/EvoZero_PAB1_GFP_task_clear/PAB1_GFP_data/PAB1.txt'

pab1_wt_sequence = (
        "GNIFIKNLHPDIDNKALYDTFSVFGDILSSKIATDENGKSKGFGFVHFEEEGAAKEAIDALNGMLLNGQEIYVAP"
    )
starts = {
        "start_seq": "GNIFIKNLHPDIDNKALYDTFSVFGDILSSKIATDENGKSKGFGFVHFEEEGAAKEAIDALKGMLLNGQEIYFAP" # noqa: E501
    }
AAS = "ILVAGMFYWEDQNHCRKSTP"


starttime = datetime.datetime.now() 
model = train_cnn_predictor(data_dir)
training_pipeline = TrainPipeline(
    starts["start_seq"], 
    AAS,
    model,
    trust_radius=100,
)
training_pipeline.run()

cmd:  nvidia-smi -q -d Memory |grep -A4 GPU|grep Free
        Free                              : 4165 MiB
        Free                              : 4213 MiB
        Free                              : 4213 MiB
        Free                              : 4253 MiB
        Free                              : 45634 MiB
        Free                              : 45634 MiB
        Free                              : 45634 MiB
        Free                              : 45634 MiB

['0:4165', '1:4213', '2:4213', '3:4253', '4:45634', '5:45634', '6:45634', '7:45634']
max remain memory GPU: 4
part:1
train_loss:0.10480683296918869
起始序列：GNIFIKNLHPDIDNKALYDTFSVFGDILSSKIATDENGKSKGFGFVHFEEEGAAKEAIDALKGMLLNGQEIYFAP


/alldata/Htang_data/workspace/project/EvoZero_PAB1_GFP_task_clear/PAB1_GFP_task/p_v_net_3.py:46: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  x_act = F.log_softmax(self.act_fc1(x_act))
/home/Htang/miniconda3/envs/lm_gvp/lib/python3.6/site-packages/torch/nn/functional.py:1698: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


AI move: 1245

move_fitness: 0.528081

episode_seq len: 2

Mmove & playout dict len: 199

GNIFIKNLHPDIDNKALYDTFSVFGDILSSKIATDENGKSKGFGFVHFEEEGAAKEAIDALKMMLLNGQEIYFAP
AI move: 170

move_fitness: 0.568954

episode_seq len: 3

Mmove & playout dict len: 398

GNIFIKNLDPDIDNKALYDTFSVFGDILSSKIATDENGKSKGFGFVHFEEEGAAKEAIDALKMMLLNGQEIYFAP
AI move: 1374

move_fitness: 0.704265

episode_seq len: 4

Mmove & playout dict len: 593

GNIFIKNLDPDIDNKALYDTFSVFGDILSSKIATDENGKSKGFGFVHFEEEGAAKEAIDALKMMLLNGCEIYFAP
AI move: 1239

move_fitness: 0.676428

episode_seq len: 5

Mmove & playout dict len: 792

GNIFIKNLDPDIDNKALYDTFSVFGDILSSKIATDENGKSKGFGFVHFEEEGAAKEAIDALPMMLLNGCEIYFAP
batch i:1, episode_len:4
起始序列：GNIFIKNLDPDIDNKALYDTFSVFGDILSSKIATDENGKSKGFGFVHFEEEGAAKEAIDALKMMLLNGCEIYFAP
AI move: 1117

move_fitness: 0.805845

episode_seq len: 6

Mmove & playout dict len: 792

GNIFIKNLDPDIDNKALYDTFSVFGDILSSKIATDENGKSKGFGFVHFEEEGAAKSAIDALKMMLLNGCEIYFAP
AI move: 136

move_fitness: 0.752828

episode_seq len: 7

Mmove &

AI move: 914

move_fitness: 1.323082

episode_seq len: 37

Mmove & playout dict len: 2989

GNIFIKNLDTDIDNKALYDTVSVFGDIMSSKIATDENGKHKGFGFCHFEEEGAAKSAIRALKMMSLNGCEIYAPQ
batch i:22, episode_len:1
kl:0.09130,lr_multiplier:0.296,loss:6.331491947174072,entropy:6.018428325653076,explained_var_old:0.002,explained_var_new:0.002
起始序列：GNIFIKNLDTDIDNKALYDTVSVFGDIMSSKIATDENGKHKGFGFVHFEEEGAAKSAIRALKMMSLNGCEIYAPQ
AI move: 1007

move_fitness: 1.427140

episode_seq len: 38

Mmove & playout dict len: 2989

GNIFIKNLDTDIDNKALYDTVSVFGDIMSSKIATDENGKHKGFGFVHFEEYGAAKSAIRALKMMSLNGCEIYAPQ
AI move: 378

move_fitness: 1.593732

episode_seq len: 39

Mmove & playout dict len: 3185

GNIFIKNLDTDIDNKALYTTVSVFGDIMSSKIATDENGKHKGFGFVHFEEYGAAKSAIRALKMMSLNGCEIYAPQ
AI move: 1022

move_fitness: 1.645805

episode_seq len: 40

Mmove & playout dict len: 3381

GNIFIKNLDTDIDNKALYTTVSVFGDIMSSKIATDENGKHKGFGFVHFEEYVAAKSAIRALKMMSLNGCEIYAPQ
AI move: 1108

move_fitness: 1.622779

episode_seq len: 41

Mmove & playout dict len: 3580

GNI

SystemExit: 0

/home/Htang/miniconda3/envs/lm_gvp/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3351: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
